In [1]:
pip install -U "autogen-agentchat"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.16.0
    Uninstalling opentelemetry-api-1.16.0:
      Successfully uninstalled opentelemetry-api-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-sdk 1.16.0 requires opentelemetry-api==1.16.0, but you have opentelemetry-api 1.31.0 which is incompatible.


In [2]:
pip install "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 13.6 MB/s eta 0:00:00


In [6]:
from google.colab import userdata
import os
api_key=userdata.get('Gemini_main')
# Set the API key as an environment variable
os.environ["GEMINI_API_KEY"] = api_key

# You can verify if the environment variable is set correctly:
#print(os.environ.get("GEMINI_API_KEY"))

In [10]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b", ## cannot use gemini-2.0-flash-lite as it is not yet compatible
    api_key=api_key,

)

response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)


finish_reason='stop' content='Paris\n' usage=RequestUsage(prompt_tokens=7, completion_tokens=2) cached=False logprobs=None thought=None


## Using a AssistantAgent

In [11]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [12]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    #tools=[],
    system_message="You are a funny assistant that answers user query in a funny way",
)

In [14]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="What is AI?", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages) ## Thinking process of the user(Used when tools are defined)
    print(response.chat_message) ## Actual Response given by the agent


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


[]
source='assistant' models_usage=RequestUsage(prompt_tokens=157, completion_tokens=134) metadata={} content='AI, or Artificial Intelligence, is basically a fancy way of saying a computer program that tries really, *really* hard to act like a human.  It\'s like teaching a parrot to mimic you, but instead of saying "Polly want a cracker," it\'s trying to write poetry, diagnose diseases, or even play chess.  The goal is to create machines that can learn, reason, and solve problems – kinda like a super-smart, silicon-based pet.  Sometimes it works brilliantly, sometimes it just says really weird things.  It\'s a bit of a wild west out there in the AI world, still figuring things out.\n' type='TextMessage'


## Using AssistantAgent with custom tool(basic)

In [15]:
# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)


In [16]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=22, completion_tokens=6), metadata={}, content=[FunctionCall(id='', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='', is_error=False)], type='ToolCallExecutionEvent')]
source='assistant' models_usage=None metadata={} content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


## Multimodal AssistantAgent

In [ ]:
import asyncio
from pathlib import Path
from autogen_agentchat.messages import MultiModalMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken, Image
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    seed=42,
    temperature=0,
)

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    model_client=model_client,
)

async def assistant_run() -> None:
    response = await assistant.on_messages(
        [
            MultiModalMessage(
                content=["Here is an image:", Image.from_file(Path("test.png"))],
                source="user",
            )
        ],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)

# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()
